# Inference and Save Model

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import torch
import os

peft_model_path = '../models/amz_movie_tv_distilgpt2_50k_random'

config = PeftConfig.from_pretrained(peft_model_path)
model = AutoModelForCausalLM.from_pretrained("distilgpt2")
model = PeftModel.from_pretrained(model, peft_model_path)
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

In [7]:
text = 'I have to'

inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], pad_token_id=tokenizer.pad_token_id, max_new_tokens=10)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I have to be careful about what they are doing. I don


In [8]:
# upload to huggingface hub
from huggingface_hub import notebook_login

notebook_login()

In [9]:
model.push_to_hub(peft_model_path.split('/')[-1], use_auth_token=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/594k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/neilsun2009/amz_movie_tv_distilgpt2_50k_random/commit/48be95d2fd404318bf1f515dab0adbcc90c78bc4', commit_message='Upload model', commit_description='', oid='48be95d2fd404318bf1f515dab0adbcc90c78bc4', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
# hugging face hosted inference API
import requests
import os


API_URL = f"https://api-inference.huggingface.co/models/neilsun2009/{peft_model_path.split('/')[-1]}"
# API_URL = f"https://api-inference.huggingface.co/models/gpt2"
headers = {"Authorization": f"Bearer !"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": "I think tha",
    # these params actually are not useable for PEFT models
    "parameters": {
        "return_full_text": False,
        "temperature": 5.0,
        "repetition_penalty": 10.0,
        "top_k": 5
    }
})

print(output)

{'error': 'Authorization header is correct, but the token seems invalid'}
